In [ ]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator")
import behresp
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [ ]:
# read in PUF
puf_df = pd.read_csv(r'C:\Users\erin.melly\Desktop\GitHub\Tax-Calculator\PUF.csv')

In [ ]:
# create new variable for gains taxed at death in PUF

# share of capital gains
puf_df = puf_df.assign(ltgains_weight = (puf_df['p23250'] * puf_df['s006']) / sum(puf_df['p23250'] * puf_df['s006']))
puf_df.loc[(puf_df['p23250'] > 0), 'ltgains_weight'] = (puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006']) \
/ sum(puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006'])
# assign total realization at death revenue from JCT to taxpayers based on capital gains weight
puf_df['gains_at_death'] = (puf_df['ltgains_weight'] * 215627184994) / (puf_df['s006'] / 100) # NOTE: change this value per JCT estimate, per year 

In [ ]:
cyr = 2022

# current law
rec = Records(data= puf_df)
pol = Policy()
calc= Calculator(pol, rec)
calc.advance_to_year(cyr)
calc.calc_all()
df = calc.dataframe([], all_vars=True)
rev1static = calc.weighted_total('combined')

### dynamic -- behavioral responses

In [ ]:
biden = {
    #restore top brackets
    "II_brk5": {2020: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00]},
    "II_rt5": {2020: 0.33},
    "II_brk6": {2020: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00]},
    "II_rt7": {2020: 0.396},
    "PT_brk5": {2020: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00]},
    "PT_rt5": {2020: 0.33},
    "PT_brk6": {2020: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00]},
    "PT_rt7": {2020: 0.396},
    # restore Pease limitations
    "ID_ps": {2020: [261500.0, 313800.0, 156900.0, 287650.0, 313800.0]},
    "ID_prt": {2020: 0.03},
    "ID_crt": {2020: 0.8},
    # Phase out section 199A for high-income
    "PT_qbid_taxinc_thd": {2020: [400000.00, 400000.00, 400000.00, 400000.00, 400000.00]},
    "PT_qbid_taxinc_gap": {2020: [100000.0, 100000.0, 100000.0, 100000.0, 100000.00]},
    # tax capital gains as ordinary income above $10 million
    "CG_brk4": {2020: [1000000, 1000000, 1000000, 1000000, 1000000]},
    "CG_rt3": {2020: 0.396},
    # capital gains realization at death
    "CG_TaxAtDeath": {2020: True},
    "CG_exclusion_thd": {2020: [100000, 200000, 100000, 100000, 200000]},
    # extend childless EITC to workers above 65 years old
    "EITC_MaxEligAge": {2020: 125},
    # limit the value of ID to 28% of value
    #"ID_Biden_Limit": {2020: True},
    "ID_BenefitSurtax_trt": {2020: 1.0},
    "ID_BenefitSurtax_crt": {2020: 0.28},
    "ID_BenefitSurtax_Switch": {2020: [True, True, False, True, True, True, True]},
    # apply earnings above $400K to OASDI tax
    "SS_Earnings_thd": {2020: 400000},
    # replace deductibility of IRA/DC pension contributions with 26% credit
    "ALD_IRAContributions_hc": {2020: 1.0},
    "ALD_KEOGH_SEP_hc": {2020: 1.0},
    "IRADC_credit_rt": {2020: 0.26},
    # reinstate energy efficiency credit
    "CR_ResidentialEnergy_hc": {2020: 1.0}
}

In [ ]:
# specify Calculator object for static analysis of reform policy
pol.implement_reform(biden)
calc2 = Calculator(pol, rec)

# calculate reform income tax liabilities for cyr under static assumptions
calc2.advance_to_year(cyr)
calc2.calc_all()
rev2static = calc2.weighted_total('combined')

# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.25,
                         'cg': -3.488}

# specify Calculator object for analysis of reform with behavioral responses
calc2 = Calculator(policy=pol, records=rec)
calc2.advance_to_year(cyr)
_, df2br = behresp.response(calc, calc2, response_elasticities)

# calculate reform income tax liabilities for cyr with behavioral response
rev2br = (df2br['combined'] * df2br['s006']).sum()

# print total income tax revenue estimates for cyr
# (estimates in billons of dollars)
print('{}_DYNAMIC_DIFERENCE_tot_rev($B)= {:.3f}'.format(cyr, (rev2br - rev1static) * 1e-9))